In [7]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os

## Preparing data as Dataframe objecs

In [2]:
path_train_rumors = "..\\data\\subtask-5A-arabic\\train_rumors.json"
path_dev_rumors = "..\\data\\subtask-5A-arabic\\dev_rumors.json"
path_relevance_judgments = "..\\data\\subtask-5A-arabic\\relevance_judgments.txt"

In [3]:
def read_json_file(path):
    f = open(path , encoding="utf8")
    data = json.load(f)
    f.close()
    return data

In [4]:
columns = ["rumor_id", "0", "user_id", "relevance"]
def text_to_dataframe(path , columns):
    df = pd.DataFrame(columns=columns)
    file = open(path , 'r')

    for line in file:
        line = line.strip().split('\t')
        df.loc[len(df)] = line

    file.close()
    
    
    return df

In [5]:
def json_to_dataframe(json_object):
    df = pd.DataFrame()
    for i in json_object[0].keys():
        df[i] = [json_object[j][i] for j in range(len(json_object))]
    return df

The metaadata directory contains **1000 json files**, each one of them contain **1193 lines**. Each line of these 1193 contains **10 features**

In [11]:
df_metadata = pd.DataFrame([])


directory_path = "..\\data\\subtask-5A-arabic\\lists_metadata" # Replace with the path to your directory
json_files = [file for file in os.listdir(directory_path) if file.endswith('.json')]

t = time.time()
for file in json_files:
    file_path = os.path.join(directory_path, file)
    df_ = json_to_dataframe(json_meta_data)
    df_metadata = pd.concat([df_, df_metadata], ignore_index=True)  
print("the time required for this operation is " , time.time()-t , " seconds")
del df_    

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\subtask-5A-arabic\\lists_metadata'

In [235]:
df_metadata.head()

,list_id,name,description,translated_name,translated_desc,member_count,follower_count,created_at,owner_id,owner_name
0,959752445068013568,محللين سياسيين,None,محللين سياسيين,None,3,0,Sat Feb 03 11:36:28 +0000 2018,2352145892,Eng : Naji ALshehab
1,956586155587358720,List1,None,القائمة 1,None,2,0,Thu Jan 25 17:54:46 +0000 2018,951082786072514561,أستاذ جامعي
2,952270776249520129,مغردون عرب,من مصر والمغرب العربي والعالم,مغردون عرب,من مصر والمغرب العربي والعالم,8,0,Sat Jan 13 20:07:00 +0000 2018,1869003181,Bu Ali-K
3,943885159228723204,First,None,أولا,None,2,0,Thu Dec 21 16:45:33 +0000 2017,831949971255590913,yemen ali
4,943555598393729024,صحفين,None,صحفين,None,6,0,Wed Dec 20 18:55:59 +0000 2017,886234134003290113,Almuraysi


## Data analysis

### 1. Train and Developpement rumors

In [212]:
train_rumors = read_json_file(path_train_rumors)
dev_rumors = read_json_file(path_dev_rumors)

train_rumors = json_to_dataframe(train_rumors)
dev_rumors = json_to_dataframe(dev_rumors)

In [213]:
train_rumors.columns , train_rumors.shape 

(Index(['rumor_id', 'tweet_id', 'tweet_text', 'category'], dtype='object'),
 (120, 4))

In [214]:
dev_rumors.columns , dev_rumors.shape

(Index(['rumor_id', 'tweet_id', 'tweet_text', 'category'], dtype='object'),
 (30, 4))

In [215]:
train_rumors.tail(3)

,rumor_id,tweet_id,tweet_text,category
117,AuFIN_118,1425132064450531328,#فيديو || اول مرّة في تاريخ الاندية الفرنسية ن...,sports
118,AuFIN_119,1221841304512057344,شوبير يعترف ان ادارة الأهلي ضغطت على الكاف واج...,sports
119,AuFIN_120,1434275913152217089,عاجل: النايل سات ١٣٠ مليون مشاهد قنوات mbc بر...,sports


In [216]:
dev_rumors.head(3)

,rumor_id,tweet_id,tweet_text,category
0,AuFIN_121,1406642665689649156,بشرى سارة للأشقاء في الخليج …\n\nالمحكمة الإسر...,politics
1,AuFIN_122,1335681592300331010,#عاجل وفاة #خامنئي المرشد الأعلى في إيران منذ ...,politics
2,AuFIN_123,1324220773435727872,#الجزائر ترفض التطبيع العلني وتفضله أن يكون سر...,politics


In [217]:
train_rumors["category"].value_counts()

politics    40
health      40
sports      40
Name: category, dtype: int64

In [218]:
dev_data["category"].value_counts()

politics    10
health      10
sports      10
Name: category, dtype: int64

In [219]:
def print_rumors(data):
    for i in data.index:
        print(data["tweet_text"][i])

In [220]:
print_rumors(train_rumors)

موسى أبو مرزوق الحمساوى يهنئ طالبان بفوزه بأفغانستان ويعتبر مندوب لملالي إيران 😉
- الصورة من إجتماع الأرهابيين وفد من طالبان مع هنية ليلة الأمس   😎
تم في كشك قطر "ممول الا.ر.هاب" 
#حفظ_الله_مصر https://t.co/Rnd14ZXv6C
مزيداً من الأنحدار نحو مستنقع التطبيع القذر..حسين الجسمي يشارك في تلحين اغنية إسرائيلية تبدأ برساله شكر للأمارات من الرئيس الأسرائيلي في تل ابيب على أنقاض قرانا الفلسطينية التي مسحت عن وجه الارض...خسئتم انتم عار علينا.. والعروبة والدين بريئةٌ من امثالكم 
#التطبيع_خيانة_عظمى https://t.co/1pdoGqkZSH
بتوجيهات من هيثم بن طارق آل سعيد سلطان عمان استدعت سلطنة عمان  سفيرها في الدوحة، اليوم  ودعت قطر إلى الكف عن نشاطاتها المزعزعة للاستقرار والأمن.
وتأتي الخطوة تعكس التضامن مع المملكة العربية السعودية التي قطعت العلاقات من قطر منتصف عام 2017 ، إضافة إلى الإمارات https://t.co/LMVgSR1OSi
.
.

متظاهرون أتراك يهاجمون البنك الكويتي التركي في إسطنبول .. بعد انهيار الليرة التركية ..

.
كما وصلني .. https://t.co/qncixBrPXg
وين ولد الملحه 
عــــــــاجل
رئيس اركان الجيش الكويتي الفريق الركن

Every single rumor in the training data belongs to one one single user.

In [221]:
a = train_rumors["tweet_id"].value_counts()
max(a)

1

In [222]:
a = dev_rumors["tweet_id"].value_counts()
max(a)

1

### 2. Relevence data

In [223]:
columns = ["rumor_id", "0", "user_id", "relevance"]
relevance_data = text_to_dataframe(path_relevance_judgments , columns).drop(["0"] , axis=1)
relevance_data.tail(3)

,rumor_id,user_id,relevance
1041,AuFIN_023,1186788943930499072,2
1042,AuFIN_102,2572368029,1
1043,AuFIN_104,827130813632376836,2


This data assigns a degree of relevence to each pair of a rumor and one of its corresponding authorities; 2 if the authority is highly relevant to the rumor (has higher priority to be contacted); 1 if she is relevant.

The number of authorities used to verify the rumors in the training and developpement data are:

In [224]:
relevance_data["user_id"].value_counts()

259372802              25
142264595              13
460210929              13
569792145              12
293466923              11
                       ..
1295065674994204672     1
2276640199              1
1335830366              1
19280428                1
827130813632376836      1
Name: user_id, Length: 526, dtype: int64

526 authorities has been conssulted to verify the rumors of the training and developpement data.

The authority with **user_id = 259372802** has been used to verify **25** rumors. 

In [225]:
relevance_data["relevance"].value_counts()

2    619
1    425
Name: relevance, dtype: int64

only pairs with relevance = 1 (425 times) or 2 (619 times) are reported in the relevance_data. Relevance = 0 is assumed for all pairs not appearing in the dataframe.

In [226]:
relevance_data["rumor_id"].value_counts()

AuFIN_106    22
AuFIN_076    19
AuFIN_098    18
AuFIN_069    17
AuFIN_100    17
             ..
AuFIN_081     1
AuFIN_118     1
AuFIN_014     1
AuFIN_146     1
AuFIN_104     1
Name: rumor_id, Length: 150, dtype: int64

The number of unique rumors in the relevence data are 150 (**120 train_rumors** and **30 dev_rumors**).

The rumor with **ID = AuFIN_106** can be verified by **22** authorities.

In [227]:
relevance_data[relevance_data["rumor_id"] == "AuFIN_106"]

,rumor_id,user_id,relevance
344,AuFIN_106,4816936572,2
345,AuFIN_106,1116034096818466816,1
346,AuFIN_106,570223846,1
347,AuFIN_106,246727008,2
348,AuFIN_106,844860412008980481,2
349,AuFIN_106,710569086296653824,1
350,AuFIN_106,891970979374694400,1
351,AuFIN_106,1099928406156816384,2
352,AuFIN_106,709085388468588544,1
353,AuFIN_106,717844112364404738,1


In [236]:
# Data represents the dataframe containing the rumor with id=AuFIN_023
data = train_rumors[train_rumors["rumor_id"] == "AuFIN_023"]
print_rumors(data)

بكل فخر صنع في مصر 🇪🇬 رامي السبيعي اول مصري واول عربي يفوز بمستر اولمبيا في التاريخ مبروك لمصر والعرب #بيج_رامي https://t.co/BzNmoR33A4


To extract the user IDs of the authorities that can be consulted to verify a specific rumor ID (e.g., **AuFIN_023**), execute the following instruction:

In [230]:
data = relevance_data[relevance_data["rumor_id"] == "AuFIN_023"]
data

,rumor_id,user_id,relevance
737,AuFIN_023,70407496,2
738,AuFIN_023,435697807,2
1041,AuFIN_023,1186788943930499072,2


The user_id in relevance_data belongs to the user_id of the twitter account of the auathority. To check the twitter accounts associated to these authorities, execute the following instruction, and use the provided link. For instance https://twitter.com/intent/user?user_id=1186788943930499072 .

In [231]:
user_id = "70407496"  #copy the user_id of the authority
link = "https://twitter.com/intent/user?user_id="+user_id
link

'https://twitter.com/intent/user?user_id=70407496'

### 3. Meta Data